**Table of contents**<a id='toc0_'></a>    
- [README](#toc1_)    
  - [環境構築とライブラリのインポート](#toc1_1_)    
  - [データベースの作成と接続](#toc1_2_)    
  - [カーソルの入手](#toc1_3_)    
  - [テーブルの作成](#toc1_4_)    
  - [データベースへの値の代入](#toc1_5_)    
  - [データベースから値の取り出し](#toc1_6_)    
  - [データベース接続を閉じる](#toc1_7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[README](#toc0_)

参考：https://docs.python.org/ja/3/library/sqlite3.html#  
SQL文の参考：https://www.javadrive.jp/sqlite/

## <a id='toc1_1_'></a>[環境構築とライブラリのインポート](#toc0_)

sqlite の python ライブラリである `sqlite3` は python 2.5 以上で標準サポートされている。  
import は以下の用に行う

In [1]:
import sqlite3


In [2]:
# type hint 用
from sqlite3 import Connection, Row, Cursor

## <a id='toc1_2_'></a>[データベースの作成と接続](#toc0_)

以下のコードでデータベースをファイルとして作成して接続する。  
既に作成済みの場合は接続する。  

In [3]:
# このノートブックを再利用するために作成されているデータベースファイルを削除するためのプログラム
from pathlib import Path
curr_dir = Path("")
for p in list(curr_dir.glob('*.sqlite3')):
    if p.exists():
        p.unlink()

In [4]:
DB_NAME = "t1.sqlite3"
conn: Connection = sqlite3.connect(DB_NAME)

## <a id='toc1_3_'></a>[カーソルの入手](#toc0_)

カーソルとは データベースからデータを取得したりするための SQL 文を実行するために必要なものです。

In [5]:
# カーソルを取得
c: Cursor = conn.cursor()

## <a id='toc1_4_'></a>[テーブルの作成](#toc0_)

データベースにデータを保存するにはまず、テーブルと呼ばれる表を作成する必要があります。  
テーブルの作成方法は Cursor インスタンスのメソッド`execute`に SQL文を渡します。  
テーブル作成のSQL文は`CREATE TABLE テーブル名 (変数名1 SQLiteの型名, 変数名2 SQLiteの型名)`です。  
SQLiteの型名とpythonの型との互換性は以下のURLを参考にしてください。  
- https://docs.python.org/ja/3/library/sqlite3.html#sqlite-and-python-types  

お試しに以下の様なテーブルを作成します。

In [6]:
# テーブルを作成
c.execute('CREATE TABLE articles  (id int, title text, body text)')

テーブルが作成されているか確認するために以下の文を実行します。  
これはデータベースに保存されているテーブル名を表示する SQL文です。  
SQL文がわからなくてもとりあえず実行してください。

In [7]:
res = c.execute("SELECT name FROM sqlite_master")
res.fetchone()

('articles',)

また以下の様に実行するとテーブルが作成された際のSQL文が把握できるので、どのような列名（以下カラム）があるかを確認できます。

In [8]:
res = c.execute("SELECT * FROM sqlite_master")
res.fetchall()

[('table',
  'articles',
  'articles',
  2,
  'CREATE TABLE articles  (id int, title text, body text)')]

## <a id='toc1_5_'></a>[データベースへの値の代入](#toc0_)

以下のSQL文を用いてデータを登録する。データベースへの保存は後述の`commit`をしなければならない。  
`INSERT INTO テーブル名 values(value1, value2, ...)`  
ここで文字列を扱う場合はシングルクウォーテーションで囲う.  


In [9]:
c.execute("INSERT INTO articles values(1, 'test1', 'this is test body')")

In [10]:
conn.commit()

以下の文でデータベースからデータを取り出して確認する

In [11]:
res = c.execute("SELECT * FROM articles")
res.fetchall()

[(1, 'test1', 'this is test body')]

またデータベースへの値の代入方法には以下のようなモノも存在する。  
- 変数からの代入
- 一回で複数のデータの登録


In [12]:
# 変数からの代入
data = [(2, 'test2', 'this is test body')]
c.executemany("INSERT INTO articles values(?, ?, ?)", data)
conn.commit()

res = c.execute("SELECT * FROM articles")
res.fetchall()

[(1, 'test1', 'this is test body'), (2, 'test2', 'this is test body')]

In [13]:
# 1回で複数のデータの登録
c.execute("INSERT INTO articles values(3, 'test3', 'this is test body'), (4, 'test4', 'this is test body')")
conn.commit()

res = c.execute("SELECT * FROM articles")
res.fetchall()

[(1, 'test1', 'this is test body'),
 (2, 'test2', 'this is test body'),
 (3, 'test3', 'this is test body'),
 (4, 'test4', 'this is test body')]

## <a id='toc1_6_'></a>[データベースから値の取り出し](#toc0_)

データベースから値を取り出すには、既出であるが、以下のSQL文を用いる  
`SELECT カラム名1, カラム名2,... FROM テーブル名`  
カラム名に対応したデータが取得でき、executeメソッドの戻り値のインスタンスの`fetchall`メソッドでタプルのリスト形式として取得することができる。  



In [14]:
res = c.execute("SELECT id, title FROM articles")
res_list = res.fetchall()
print(res_list)

# また、カラム名に`*`を使用することで全てのカラム名を対象にすることができる。  
res = c.execute("SELECT * FROM articles")
res_list = res.fetchall()
print(res_list)

[(1, 'test1'), (2, 'test2'), (3, 'test3'), (4, 'test4')]
[(1, 'test1', 'this is test body'), (2, 'test2', 'this is test body'), (3, 'test3', 'this is test body'), (4, 'test4', 'this is test body')]


## <a id='toc1_7_'></a>[データベース接続を閉じる](#toc0_)

データベースの接続を閉じるには以下のように実行する。  
プログラムの終了時に必ずつけた方がよいと思う。  

In [15]:
conn.close()